# Get tf-idf_SVD, euclidean-distance and cosine-distance matrices
The overall goal for this script is to iterate through **neuroscience-related R01s** for **each year**, get the abstracts for each of those projects and return three matrices and one dataframe:
1. **tf-idf_SVD**: This is a SVD-decomposed tf-idf wordcount matrix [n_abstracts, n_components]. I will use this for k-means clustering and affinity propagation (I think).
2. **cos_SVD**: This is a cosine distance matrix [n_abstracts, n_abstracts] generated from tf-idf_SVD. I will use this for Ward hierarchical clustering.
3. **euc_SVD**: This is an euclidean distance matrix [n_abstracts, n_abstracts] generated from tf-idf_SVD. I will also use this for Ward hierarchical clustering.
4. **df_neuro**: This is a dataframe, filtered for neuro-related R01s, containing information about each project.

In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot

from string import punctuation
from nltk.corpus import stopwords
from nltk import PorterStemmer

%matplotlib inline

# Read abstracts and project csvs into dataframes

In [2]:
initialize_df = pd.DataFrame(columns=['wordlist', 'year'])

In [3]:
try: 
    initialize_df.to_csv('all_years.csv')
except: 
    pass

In [4]:
def get_abstracts_projects(year):
    """
    input: int -- year to analyze; [1985:2016] inclusive.
    returns: 2 dataframes -- project & abstract
    """
    abs_dir = './abstracts/'
    proj_dir = './projects/'
    
    project = pd.read_csv(proj_dir + 'RePORTER_PRJ_C_FY' + year + '.csv', encoding = "ISO-8859-1")
    abstract = pd.read_csv(abs_dir + 'RePORTER_PRJABS_C_FY' + year + '.csv',encoding = "ISO-8859-1")
    
    return project, abstract

# Clean and filter for neuro abstracts

### Clean abstracts (stemming, lowercase, no punctuation, remove stopwords)

In [5]:
# get customized stopwords:
def customized_stopwords(to_append=[]):
    """
    to_append: list; what words do you want to exclude from your analysis, in addition to the standard 
    stopwords like 'the', 'and, 'of', and so on? See above to_append variable for examples.
    returns: list; stopwords including to_append list
    """
    stop = stopwords.words('english')
    stop = stop + to_append
    return stop

# get list of words that are lowercase, with punctuation removed and words stemmed.
def get_wordlist(abstract, stop = customized_stopwords()):
    """
    returns a list of lowercase words from abstract with punctuation and stopwords removed.
    """
    try:
        # make words lowercase
        words = abstract.lower()
        
        # take out all punctuation and split strings into a list of words
        words = (''.join(c for c in words if c not in punctuation)).split(' ')
        
        # remove stopwords
        words = [" ".join([w for w in word.split() if not w in stop]) for word in words]
        
        # stem words using Porter's Stemmer
        stemmed = []
        for word in words:
            try:
                word = PorterStemmer().stem(word)
            except IndexError:
                word = word
            if word != '' and word.isalpha():
                stemmed.append(word)
        words = stemmed

    except AttributeError:
        words = []
    return words

### Filter for neuro-related abstracts.
I defined a project to be neuroscience-related if the abstract mentioned "brain" or "neur*" at least once every 100 non-stopwords.

In [6]:
def neuro_count(row):
    try:
        return (row.ABSTRACT_TEXT.count(' brain') + row.ABSTRACT_TEXT.count('neur'))
    except AttributeError:
        return 0
    
def wordlist_count(row):
    return len(row.wordlist)
    
def neuro_only(df, word_density=0.01):
    """
    input: dataframe
    word_density: how many neuro related words for every 100 words that are not stopwords in an abstract? Stopwords: the, and, or, not, etc.
    returns: dataframe containing neuro-related projects as defined above, with a column containing cleaned abstract keywords for analysis.
    """
    df['abs_neuro_count'] = df.apply(neuro_count, axis=1)
    df['wordlist'] = df.ABSTRACT_TEXT.apply(get_wordlist)
    df['wordlist_ct'] = df.apply(wordlist_count, axis=1)
    df['rel_neuro_count'] = df.abs_neuro_count / df.wordlist_ct
    
    # drop duplicates - I found that in some years, there are duplicate abstracts which really screwed up my hierarchical clustering
    df = df.drop_duplicates('ABSTRACT_TEXT')
    return df[df.rel_neuro_count >= word_density]

# Implementation

In [9]:
# years = range(1985,2017)
years = [1996]
grant_type = 'R01'

# get abstracts and projects for a single year
for year in years:
    project, abstract = get_abstracts_projects(str(year))

    # join abstracts to projects dataframe
    df = project.merge(abstract, on='APPLICATION_ID', how='left')
    
    # Implement clean abstracts and filter for neuro projects.
    df_neuro = neuro_only(df)

    # look only at R01s
    df_neuro_granttype = df_neuro[df_neuro.ACTIVITY == grant_type]

    df_to_save = pd.DataFrame(df_neuro_granttype['wordlist'])
    df_to_save['year'] = year

#     df_to_save.to_csv('all_years.csv', mode='a', header=False)
    
    print (year, 'done')
    
print ('all done')

//anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2825: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


1996 done
all done


In [22]:
project.shape

(65224, 42)

In [21]:
project[project.ACTIVITY=='R01']

,APPLICATION_ID,ACTIVITY,ADMINISTERING_IC,APPLICATION_TYPE,ARRA_FUNDED,AWARD_NOTICE_DATE,BUDGET_START,BUDGET_END,CFDA_CODE,CORE_PROJECT_NUM,...,PROJECT_TERMS,PROJECT_TITLE,SERIAL_NUMBER,STUDY_SECTION,STUDY_SECTION_NAME,SUBPROJECT_ID,SUFFIX,SUPPORT_YEAR,TOTAL_COST,TOTAL_COST_SUB_PROJECT
29770,2042885,R01,AA,5,NaN,1996-05-24T00:00:00,06/01/1996,03/31/1999,273.0,R01AA000186,...,aldehyde dehydrogenases;isozymes;enzyme mechan...,ALDEHYDE DEHYDROGENASE ISOZYMES AND ALCOHOLISM,186,ALCB,Alcohol Biomedical Research Review Committee,NaN,NaN,23.0,NaN,NaN
29771,2042892,R01,AA,5,NaN,1996-01-29T00:00:00,02/01/1996,01/31/1997,273.0,R01AA000187,...,NaN,COGNITIVE DEFICITS RELATED TO CHRONIC ALCOHOLISM,187,ALCB,Alcohol Biomedical Research Review Committee,NaN,NaN,24.0,NaN,NaN
29772,2042949,R01,AA,5,NaN,1995-11-29T00:00:00,12/01/1995,11/30/1996,273.0,R01AA000279,...,NaN,SPECIFICITY OF ALCOHOL DEHYDROGENASES,279,ALCB,Alcohol Biomedical Research Review Committee,NaN,NaN,23.0,NaN,NaN
29773,2042954,R01,AA,5,NaN,1995-12-15T00:00:00,01/01/1996,12/31/1996,273.0,R01AA000626,...,NaN,HORMONAL REGULATION OF ETHANOL METABOLISM,626,ALCB,Alcohol Biomedical Research Review Committee,NaN,NaN,22.0,NaN,NaN
29774,2042961,R01,AA,5,NaN,1996-03-15T00:00:00,04/01/1996,03/31/1997,273.0,R01AA001455,...,NaN,ALCOHOL INTAKE DURING PREGNANCY--OFFSPRING DEV...,1455,ALCP,Alcohol Psychosocial Research Review Committee,NaN,NaN,21.0,NaN,NaN
29775,2042966,R01,AA,5,NaN,1995-12-15T00:00:00,01/01/1996,12/31/1996,273.0,R01AA002054,...,NaN,ALCOHOL-MEMBRANE INTERACTION IN THE BRAIN--AGI...,2054,ALCB,Alcohol Biomedical Research Review Committee,NaN,NaN,16.0,NaN,NaN
29776,2043006,R01,AA,5,NaN,1996-03-18T00:00:00,04/01/1996,03/31/1997,273.0,R01AA003312,...,NaN,ETHANOL AND ACETALDEHYDE EFFECTS ON LIVER FUNC...,3312,ALCB,Alcohol Biomedical Research Review Committee,NaN,NaN,17.0,NaN,NaN
29777,2043012,R01,AA,5,NaN,1996-08-20T00:00:00,09/01/1996,03/31/1998,273.0,R01AA003490,...,NaN,MATERNAL ALCOHOLISM AND CNS DEVELOPMENT OF OFF...,3490,SRCA,NaN,NaN,NaN,17.0,NaN,NaN
29778,2043040,R01,AA,5,NaN,1996-07-17T00:00:00,08/01/1996,07/31/1997,273.0,R01AA003624,...,NaN,CONTROL OF DRUG AND ETHANOL METABOLISM,3624,ALCB,Alcohol Biomedical Research Review Committee,NaN,NaN,18.0,NaN,NaN
29779,2043041,R01,AA,3,NaN,1996-07-19T00:00:00,08/01/1996,07/31/1997,273.0,R01AA003624,...,NaN,CONTROL OF DRUG AND ETHANOL METABOLISM,3624,ALCB,Alcohol Biomedical Research Review Committee,NaN,S1,18.0,NaN,NaN


In [24]:
abstract.shape

(26602, 2)

In [27]:
df.head()

,APPLICATION_ID,ACTIVITY,ADMINISTERING_IC,APPLICATION_TYPE,ARRA_FUNDED,AWARD_NOTICE_DATE,BUDGET_START,BUDGET_END,CFDA_CODE,CORE_PROJECT_NUM,...,SUBPROJECT_ID,SUFFIX,SUPPORT_YEAR,TOTAL_COST,TOTAL_COST_SUB_PROJECT,ABSTRACT_TEXT,abs_neuro_count,wordlist,wordlist_ct,rel_neuro_count
0,2056405,A03,AH,1,NaN,1996-04-24T00:00:00,07/01/1996,06/30/1997,NaN,A03AH001151,...,NaN,NaN,1.0,NaN,NaN,NaN,0,[],0,NaN
1,2056406,A03,AH,1,NaN,1996-04-24T00:00:00,07/01/1996,06/30/1997,NaN,A03AH001152,...,NaN,NaN,1.0,NaN,NaN,NaN,0,[],0,NaN
2,2056407,A03,AH,1,NaN,1996-04-24T00:00:00,07/01/1996,06/30/1997,NaN,A03AH001153,...,NaN,NaN,1.0,NaN,NaN,NaN,0,[],0,NaN
3,2056408,A03,AH,1,NaN,1996-04-24T00:00:00,07/01/1996,06/30/1997,NaN,A03AH001154,...,NaN,NaN,1.0,NaN,NaN,NaN,0,[],0,NaN
4,2056409,A03,AH,1,NaN,1996-04-24T00:00:00,07/01/1996,06/30/1997,NaN,A03AH001155,...,NaN,NaN,1.0,NaN,NaN,NaN,0,[],0,NaN


In [25]:
df.shape

(65224, 47)

In [29]:
df.ABSTRACT_TEXT.dropna().shape

(22124,)

In [34]:
df[df.ACTIVITY=='R01']

,APPLICATION_ID,ACTIVITY,ADMINISTERING_IC,APPLICATION_TYPE,ARRA_FUNDED,AWARD_NOTICE_DATE,BUDGET_START,BUDGET_END,CFDA_CODE,CORE_PROJECT_NUM,...,SUBPROJECT_ID,SUFFIX,SUPPORT_YEAR,TOTAL_COST,TOTAL_COST_SUB_PROJECT,ABSTRACT_TEXT,abs_neuro_count,wordlist,wordlist_ct,rel_neuro_count
29770,2042885,R01,AA,5,NaN,1996-05-24T00:00:00,06/01/1996,03/31/1999,273.0,R01AA000186,...,NaN,NaN,23.0,NaN,NaN,The long term aim of this proposal is to obtai...,1,"[long, term, aim, propos, obtain, better, unde...",202,0.004950
29771,2042892,R01,AA,5,NaN,1996-01-29T00:00:00,02/01/1996,01/31/1997,273.0,R01AA000187,...,NaN,NaN,24.0,NaN,NaN,NaN,0,[],0,NaN
29772,2042949,R01,AA,5,NaN,1995-11-29T00:00:00,12/01/1995,11/30/1996,273.0,R01AA000279,...,NaN,NaN,23.0,NaN,NaN,NaN,0,[],0,NaN
29773,2042954,R01,AA,5,NaN,1995-12-15T00:00:00,01/01/1996,12/31/1996,273.0,R01AA000626,...,NaN,NaN,22.0,NaN,NaN,NaN,0,[],0,NaN
29774,2042961,R01,AA,5,NaN,1996-03-15T00:00:00,04/01/1996,03/31/1997,273.0,R01AA001455,...,NaN,NaN,21.0,NaN,NaN,NaN,0,[],0,NaN
29775,2042966,R01,AA,5,NaN,1995-12-15T00:00:00,01/01/1996,12/31/1996,273.0,R01AA002054,...,NaN,NaN,16.0,NaN,NaN,NaN,0,[],0,NaN
29776,2043006,R01,AA,5,NaN,1996-03-18T00:00:00,04/01/1996,03/31/1997,273.0,R01AA003312,...,NaN,NaN,17.0,NaN,NaN,NaN,0,[],0,NaN
29777,2043012,R01,AA,5,NaN,1996-08-20T00:00:00,09/01/1996,03/31/1998,273.0,R01AA003490,...,NaN,NaN,17.0,NaN,NaN,NaN,0,[],0,NaN
29778,2043040,R01,AA,5,NaN,1996-07-17T00:00:00,08/01/1996,07/31/1997,273.0,R01AA003624,...,NaN,NaN,18.0,NaN,NaN,NaN,0,[],0,NaN
29779,2043041,R01,AA,3,NaN,1996-07-19T00:00:00,08/01/1996,07/31/1997,273.0,R01AA003624,...,NaN,S1,18.0,NaN,NaN,NaN,0,[],0,NaN


In [33]:
df[df.ACTIVITY=='R01'].ABSTRACT_TEXT.dropna().shape

(1068,)